In [1]:
### mmdetection에 필요한 좌표 형식 (coco dataset 기준) [top left x position, top left y position, width, height]
### class  
### 01_ulcer,02_mass,04_lymph,05_bleeding 총 4 개  

In [2]:
from mmdet.apis import init_detector, inference_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed

from custom_data_pipeline import CUSTOM_LoadImageFromFile


import json

In [3]:
@DATASETS.register_module()
class lesion_ds(CustomDataset):
    CLASSES=('01_ulcer','02_mass','04_lymph', '05_bleeding')


    def load_annotations(self, ann_file):
        
        CLASSES_dict = { '01_ulcer' : 0 , '02_mass' : 1, '04_lymph' : 2, '05_bleeding' : 3}
        
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
        
        data_infos = []
        
        for idx,img in enumerate(image_list):
            json_data = {}
            with open(img, "r") as json_file:
                json_data = json.load(json_file)
            
            filename = img # json에 annotation + image라서 json 자체를 filename로 주고 LoadImageFromFile을 baseline을 참조하여 custom으로 

            height = json_data['imageHeight']
            width = json_data['imageWidth']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for a_idx in range(len(json_data['shapes'])):
                gt_labels.append(CLASSES_dict[json_data['shapes'][a_idx]['label']])
                
                ## 좌표순서 좌상 우상 우하 좌하 
                ## mmdetection의 default annotation loader는 'coco_panoptic.py'에 있는데 여기 기준으로 하면 bbox에는 x1,y1,x2,y2가 담겨야함
                ## 만약에 이렇게 따로 load를 하지 않을시에는 x,y,w,h 형태로 annotaion을 생성하면 자동으로 x1,y1,x2,y2로 변환해줌 
                ori_pos = np.array(json_data['shapes'][a_idx]['points'])
                x1, y1, x2, y2 = min(ori_pos[:, 0]), min(ori_pos[:, 1]), max(ori_pos[:, 0]), max(ori_pos[:, 1])
                
                if x1 == 0 : 
                    x1 = 1 
                if y1 == 0 : 
                    y1 = 1 
                
                if x2 == width: 
                    x2 = x2 - 1 
                if y2 == width: 
                    y2 = y2-1 
            
                
                if x1==x2 or y1==y2:
                    print('Grond-truth Bounding Box 이상체크')
                    print(filename)
                     
                
                gt_bboxes.append([x1,y1,x2,y2])
                

            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(image_list))+' load annotations END!')
            
        
        
        return data_infos
    

In [4]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('mmdetection/configs/resnest/cascade_rcnn_s101_fpn_syncbn-backbone+head_mstrain-range_1x_coco.py') 

In [5]:
cfg.dataset_type  = 'lesion_ds'
cfg.data_root = ''

## coco datset으로 pretrain된 weight load 
# 최초학습시 pretrain path 
cfg.load_from = 'pretrain/cascade_rcnn_s101_fpn_syncbn-backbone+head_mstrain-range_1x_coco.pth'
# 12epoch까지 갔는데 아직 validation 성능이 계속오르기만해서 weight load이후 추가 학습진행 
# cfg.load_from = 'lesion_checkpoints_ver2/epoch_12.pth'
# 추가학습이후 경향이 좋아지지 않아서 학습멈춤.. 타 모델 학습시켜서 ENSEMBLE에 사용해볼 예정 
cfg.work_dir = 'lesion_checkpoints_ver2'

cfg.evaluation.metric = 'mAP'

#sync bn하면 single gpu라 error 발생 
cfg.norm_cfg = dict(type='BN', requires_grad=True)

#class 4개로 
cfg.model.roi_head.bbox_head[0].num_classes=4

cfg.model.roi_head.bbox_head[1].num_classes=4

cfg.model.roi_head.bbox_head[2].num_classes=4

#데이터 pipeline은 config 참조 
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

## 원본이 576 / 576
## 이미지의 높이 너비 비율은 동일하게 두는 것으로..
## flip 외에도   PhotoMetricDistortion  augmentation 추가..
## auto augment는 컴퓨터 자원 문제로 배제함..
cfg.train_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile',to_float32=True),
    dict(type='PhotoMetricDistortion'),
    dict(
        type='LoadAnnotations',
        with_bbox=True,
        with_mask=False,
        poly2mask=False),
    dict(
        type='Resize',
        img_scale=[(576, 576), (600, 600)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]

## 실제 test시에는 Test Time Augmentation 적용하는 것으로 
cfg.test_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(600, 600),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

# batch size 3060에 맞게 set (기존 pretrain시에는  batch 32 사용했다고 되어있음)
cfg.data = dict(
    samples_per_gpu=4,
    workers_per_gpu=2,
    train=dict(filter_empty_gt=False, pipeline=cfg.train_pipeline,ann_file='splits/lesion_train.txt'),
    val=dict(pipeline=cfg.test_pipeline,ann_file='splits/lesion_val.txt'),
    test=dict(pipeline=cfg.test_pipeline))


cfg.data.train.type = cfg.dataset_type
cfg.data.val.type = cfg.dataset_type
cfg.data.test.type = cfg.dataset_type

# optimizer AdamW로 변경이후 learning rate도 낮게 줌
cfg.optimizer = dict(type='AdamW', lr=0.0001, betas=(0.9, 0.999), weight_decay=0.05)
cfg.optimizer_config = dict(grad_clip=dict(max_norm=35, norm_type=2))

# 처음학습시에는 쓰고 12epoch에서 더 학습시킬때는 안씀..
# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=2500,
#     warmup_ratio=0.000125,
#     step=[8, 11])


#2480
cfg.log_config.interval = 500 #iteration 단위
cfg.evaluation.interval = 1 #epoch 단위
cfg.checkpoint_config.interval = 1 #epoch 단위

# seed set
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='ResNeSt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://resnest101'),
        stem_channels=128,
        radix=2,
        reduction_factor=4,
        avg_down_stride=True),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=

In [6]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import torch

In [7]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


20000/59491 load annotations END!
40000/59491 load annotations END!


In [ ]:
print(datasets)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

train_detector(model, datasets, cfg, distributed=False, validate=True)